In [1]:
import requests
import re
import json
import pandas as pd
import time
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

In [2]:
# 1. Obtener el HTML de la página del ranking
load_dotenv()
url = os.getenv("TARGET_URL_UNEMPLOYMENT_RATE")
BASE_URL = os.getenv("BASE_URL")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Lanza error si la petición falla

In [3]:
# 2. Parsear HTML con BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
# 3. Buscar la tabla principal y extraer los links
table = soup.find("table")
country_links = []


if table:
    rows = table.find_all("tr")
    for row in rows:
        cell = row.find("td")
        if cell and cell.a:
            relative_link = cell.a["href"]
            full_url = f"{BASE_URL}{relative_link}"
            country_links.append(full_url)

# Convertir la lista en DataFrame
df_links = pd.DataFrame(country_links, columns=["url"])

# Guardar como archivo Parquet
df_links.to_parquet("data/country_private_unemployment_links.parquet", index=False)

In [5]:
# 4. Mostrar resultado
print("Links encontrados:", df_links.size)

Links encontrados: 106


In [6]:
# configuracion necesaria para descargar datos

master_df = pd.DataFrame()

pattern = r'arrayToDataTable\((\[\[.*?\]\])\)'

sleep_duration = 5

In [7]:
# Cargar los países válidos desde el archivo parquet
df_countries = pd.read_parquet("data/countries_with_hpi.parquet")
valid_countries = set(df_countries["country"].unique())

In [8]:
start_time = time.time()
for url in country_links:
    # Extraer país desde el URL
    country = urlparse(url).path.strip("/").split("/")[0]

    # Verificar si el país está en la lista válida
    if country not in valid_countries:
        print(f"⏭️ País {country} no está en la lista, se omite.")
        continue

    try:
        response = requests.get(url, headers=headers)
        html = response.text

        match = re.search(pattern, html, re.DOTALL)
        if match:
            data_str = match.group(1)
            data = json.loads(data_str)

            df = pd.DataFrame(data[1:], columns=data[0])
            df.columns = ["Period", "Value"]
            df["Country"] = country

            df_pivot = df.pivot(index="Period", columns="Country", values="Value")
            master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

            print(f"✅ Datos extraídos correctamente para {country}")
        else:
            print(f"⚠️ No se encontraron datos en {url}")
    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")
    
    time.sleep(sleep_duration)

end_time = time.time()
total_sleep_time = len(valid_countries) * sleep_duration
effective_time = end_time - start_time - total_sleep_time

print(f"⏳ Tiempo total (incluyendo esperas): {end_time - start_time:.2f} s")
print(f"😴 Tiempo en espera: {total_sleep_time:.2f} s")
print(f"⚡ Tiempo efectivo de ejecución: {effective_time:.2f} s")

⏭️ País Albania no está en la lista, se omite.
⏭️ País Algeria no está en la lista, se omite.
⏭️ País Andorra no está en la lista, se omite.
⏭️ País Angola no está en la lista, se omite.
⏭️ País Argentina no está en la lista, se omite.
⏭️ País Armenia no está en la lista, se omite.
✅ Datos extraídos correctamente para Australia
✅ Datos extraídos correctamente para Austria
⏭️ País Azerbaijan no está en la lista, se omite.
⏭️ País Barbados no está en la lista, se omite.
⏭️ País Belarus no está en la lista, se omite.
✅ Datos extraídos correctamente para Belgium
⏭️ País Bolivia no está en la lista, se omite.
⏭️ País Bosnia-and-Herzegovina no está en la lista, se omite.
✅ Datos extraídos correctamente para Brazil
✅ Datos extraídos correctamente para Bulgaria
✅ Datos extraídos correctamente para Canada
✅ Datos extraídos correctamente para Chile
✅ Datos extraídos correctamente para China
✅ Datos extraídos correctamente para Colombia
⏭️ País Costa-Rica no está en la lista, se omite.
✅ Datos ex

In [14]:
master_df.tail()

,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
Period,,,,,,,,,,,,,,,,,,,,,
Sep 2020,6.72,6.0,6.4,14.9,5.8,8.4,12.35,NaN,15.77,7.6,...,4.9,NaN,3.6,16.4,8.4,5.2,NaN,12.4,7.8,5.2
Sep 2021,4.56,4.6,6.3,12.7,4.3,6.5,8.41,NaN,11.96,6.4,...,4.5,NaN,2.7,14.4,8.4,4.9,NaN,11.1,4.7,4.3
Sep 2022,3.48,4.9,5.6,8.7,3.4,4.8,8.04,NaN,10.75,6.4,...,3.8,NaN,2.4,13.0,6.5,4.0,NaN,9.9,3.5,3.8
Sep 2023,3.45,5.1,5.6,7.7,3.9,5.2,8.91,NaN,9.25,5.6,...,3.7,NaN,2.3,12.1,7.7,4.4,NaN,8.9,3.8,4.0
Sep 2024,3.89,5.4,6.0,6.4,3.5,6.0,8.74,NaN,9.13,4.6,...,4.3,NaN,2.1,11.1,8.2,4.8,NaN,8.6,4.1,4.3


In [10]:
# obtain paraguay url
template_url = country_links[0]
country = "Paraguay"

# Separar por "/"
parts = template_url.strip("/").split("/")

# Reemplazar el país (posición 3: después del dominio)

parts[3] = country

# Reconstruir la URL
custom_url = "/".join(parts) + "/"

In [26]:
# URL específica para Paraguay
response = requests.get(custom_url, headers=headers)
html = response.text

# Buscar el bloque JSON dentro del HTML
match = re.search(pattern, html, re.DOTALL)
if match:
    data_str = match.group(1)
    data = json.loads(data_str)

    # Crear DataFrame con los datos
    df = pd.DataFrame(data[1:], columns=data[0])
    df.columns = ["Period", "Value"]
    df["Country"] = "Paraguay"  # Asignar explícitamente

    # Pivotear para formato ancho (opcional)
    df_paraguay = df.pivot(index="Period", columns="Country", values="Value")

    print("✅ Datos de Paraguay extraídos correctamente")
else:
    print("⚠️ No se encontraron datos en el HTML para Paraguay")

✅ Datos de Paraguay extraídos correctamente


In [25]:
del df_paraguay

In [27]:
df_paraguay.tail()

Country,Paraguay
Period,
Q4 2020,7.17
Q4 2021,6.81
Q4 2022,5.80
Q4 2023,5.23
Q4 2024,4.55


In [17]:
base_df = master_df.copy()

In [18]:
base_df.head()

,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
Period,,,,,,,,,,,,,,,,,,,,,
Apr 1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,1.6
Apr 1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.3
Apr 1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.6,1.7
Apr 1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.7,2.3
Apr 1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.3,1.6


In [32]:
# Resetear el índice para separar "Period"
master_df = base_df.copy()
master_df = master_df.reset_index()

# Mapeo de mes a trimestre
month_to_quarter = {
    "Jan": "Q1", "Feb": "Q1", "Mar": "Q1",
    "Apr": "Q2", "May": "Q2", "Jun": "Q2",
    "Jul": "Q3", "Aug": "Q3", "Sep": "Q3",
    "Oct": "Q4", "Nov": "Q4", "Dec": "Q4"
}

# Extraer mes y año desde "Period"
master_df[["Month", "Year"]] = master_df["Period"].str.extract(r"(\w{3})\s+(\d{4})")

# Filtrar filas válidas (donde se extrajo correctamente el año)
master_df = master_df[master_df["Year"].notna()].copy()

# Mapear trimestre y convertir año
master_df["Quarter"] = master_df["Month"].map(month_to_quarter)
master_df["Year"] = master_df["Year"].astype(int)

# Reordenar las columnas: primero Quarter y Year
cols = ["Quarter", "Year"] + [col for col in master_df.columns if col not in ["Period", "Month", "Quarter", "Year"]]
master_df = master_df[cols]

# Ordenar por Year y Quarter
master_df = master_df.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [33]:
master_df.tail()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
781,Q1,2025,4.52,6.0,6.1,6.5,4.0,7.1,8.02,NaN,...,4.0,NaN,3.7,11.2,10.4,4.7,NaN,9.0,4.0,4.4
782,Q1,2025,4.31,6.0,6.2,7.0,3.8,7.1,8.71,NaN,...,4.0,NaN,3.1,11.4,8.5,4.7,NaN,8.0,4.2,4.6
783,Q2,2025,4.15,5.5,6.3,6.6,3.7,7.0,NaN,NaN,...,3.8,NaN,2.9,10.7,8.9,NaN,NaN,NaN,4.2,NaN
784,Q2,2025,4.04,5.0,6.2,6.2,3.7,7.0,NaN,NaN,...,3.7,NaN,2.8,10.2,9.7,NaN,NaN,NaN,4.2,NaN
785,NaN,2025,4.19,5.5,6.3,5.8,3.7,6.5,NaN,NaN,...,3.6,NaN,2.8,10.0,9.4,NaN,NaN,NaN,4.1,NaN


In [28]:
# Resetear el índice para trabajar con "Period"
df_paraguay = df_paraguay.reset_index()

# Extraer Quarter y Year
df_paraguay[["Quarter", "Year"]] = df_paraguay["Period"].str.extract(r"Q(\d)\s+(\d{4})").astype(int)

# Reordenar columnas: Quarter, Year, Paraguay
df_paraguay = df_paraguay[["Quarter", "Year", "Paraguay"]]

# Ordenar por Year y Quarter
df_paraguay = df_paraguay.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [34]:
df_paraguay.tail()

Country,Quarter,Year,Paraguay
27,4,2023,5.23
28,1,2024,6.90
29,2,2024,6.43
30,3,2024,5.34
31,4,2024,4.55


In [35]:
df_paraguay["Quarter"] = "Q" + df_paraguay["Quarter"].astype(str)

In [36]:
df_paraguay.tail()

Country,Quarter,Year,Paraguay
27,Q4,2023,5.23
28,Q1,2024,6.90
29,Q2,2024,6.43
30,Q3,2024,5.34
31,Q4,2024,4.55


In [38]:
# Unir por Quarter y Year
df_merged = pd.merge(master_df, df_paraguay, on=["Quarter", "Year"], how="outer")

In [39]:
df_merged.tail()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
781,NaN,2021,4.87,5.8,6.0,14.2,5.1,7.4,9.50,NaN,...,NaN,3.8,15.1,10.5,4.8,NaN,10.4,5.9,4.6,NaN
782,NaN,2022,3.50,4.0,5.6,9.3,4.2,4.7,7.81,NaN,...,NaN,3.0,12.4,8.6,3.9,NaN,9.7,3.6,3.6,NaN
783,NaN,2023,3.40,4.7,5.3,8.0,4.5,5.2,8.53,NaN,...,NaN,2.7,11.4,9.3,3.7,NaN,9.0,3.6,4.3,NaN
784,NaN,2024,3.91,4.8,5.3,6.9,3.9,6.2,8.35,NaN,...,NaN,2.9,11.1,9.4,4.0,NaN,8.9,4.1,4.2,NaN
785,NaN,2025,4.19,5.5,6.3,5.8,3.7,6.5,NaN,NaN,...,NaN,2.8,10.0,9.4,NaN,NaN,NaN,4.1,NaN,NaN


In [40]:
# Guardar como archivo Parquet
df_merged.to_parquet("data/unemployment_rates_values.parquet", index=False)